In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES']='3'

### Load FaithDial Data

In [4]:
# !pip install datasets

In [7]:
from datasets import load_dataset
dataset = load_dataset("McGill-NLP/FaithDial")

/home/csgrad/jayashok/miniconda3/envs/fact_hallucination/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: faith_dial/plain_text
Found cached dataset faith_dial (/home/csgrad/jayashok/.cache/huggingface/datasets/McGill-NLP___faith_dial/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a)
100%|██████████| 7/7 [00:00<00:00, 207.70it/s]


In [8]:
dataset

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1666
    

In [9]:
dataset['train'][0]

{'dialog_idx': 0,
 'response': 'Yeah, but once the access to the internet was a rare thing. do you remember?',
 'original_response': "No I could not! I couldn't imagine living when internet access was rare and very few people had it!",
 'history': ['Can you imagine the world without internet access?'],
 'knowledge': 'Internet access was once rare, but has grown rapidly.',
 'BEGIN': ['Hallucination'],
 'VRM': ['Disclosure', 'Ack.']}

In [39]:
import pandas as pd

In [40]:
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

In [41]:
train_df

,dialog_idx,response,original_response,history,knowledge,BEGIN,VRM
0,0,"Yeah, but once the access to the internet was ...",No I could not! I couldn't imagine living when...,[Can you imagine the world without internet ac...,"Internet access was once rare, but has grown r...",[Hallucination],"[Disclosure, Ack.]"
1,0,"Well, I know that more people started using it...","It used to be restricted, but around 1995, the...",[Can you imagine the world without internet ac...,Use by a wider audience only came in 1995 when...,"[Hallucination, Entailment]",[Edification]
2,0,Yeah. And the Internet actually developed from...,"Yes, it was developed from a government funded...",[Can you imagine the world without internet ac...,"The Internet developed from the ARPANET, which...","[Hallucination, Entailment]","[Disclosure, Ack.]"
3,0,"Well, I know that it is defined as the ability...",What is your favorite thing to do with interne...,[Can you imagine the world without internet ac...,Internet access is the ability of individuals ...,"[Hallucination, Entailment]","[Disclosure, Question]"
4,1,I am a bot so that is not posible. Do you perf...,"Yes, I perform administrative duties as a phar...",[I am just finishing my university course and ...,They may also perform administrative duties in...,"[Hallucination, Entailment]","[Disclosure, Ack.]"
...,...,...,...,...,...,...,...
18352,4093,Neat! So you raise and breed pigs?,That's pretty neat. Pig farming is raising and...,[I live on a farm with pigs!],Pig farming is the raising and breeding of dom...,"[Hallucination, Entailment]","[Disclosure, Ack.]"
18353,4093,"Well, farm pigs are raised to be eaten.",Sounds like a great cycle. Pigs are farmed mos...,"[I live on a farm with pigs!, Neat! So you rai...",Pigs are farmed principally to be eaten (e.g.,"[Hallucination, Entailment]",[Disclosure]
18354,4093,"I see. Well, historically, pigs in farms were ...",Historically farm pigs were kept in small numb...,"[I live on a farm with pigs!, Neat! So you rai...","Historically, farm pigs were kept in small num...","[Hallucination, Entailment]",[Disclosure]
18355,4093,Have you ever used your pig farm to dispose of...,Another use for pigs is to dispose of municipa...,"[I live on a farm with pigs!, Neat! So you rai...",Pigs have been farmed to dispose of municipal ...,"[Hallucination, Entailment]",[Disclosure]


In [42]:
train_df['prompt'] = train_df['history'].apply(lambda my_list: my_list[-1])
val_df['prompt'] = val_df['history'].apply(lambda my_list: my_list[-1])
test_df['prompt'] = test_df['history'].apply(lambda my_list: my_list[-1])

In [ ]:
train_df.isna().any(), test_df.isna().any(), val_df.isna().any()

(dialog_idx           False
 response             False
 original_response     True
 history              False
 knowledge            False
 BEGIN                False
 VRM                  False
 prompt               False
 dtype: bool,
 dialog_idx           False
 response             False
 original_response     True
 history              False
 knowledge            False
 BEGIN                False
 VRM                  False
 prompt               False
 dtype: bool,
 dialog_idx           False
 response             False
 original_response     True
 history              False
 knowledge            False
 BEGIN                False
 VRM                  False
 prompt               False
 dtype: bool)

In [ ]:
train_df.to_csv('train_data.csv', index=False, sep='|')
val_df.to_csv('val_data.csv', index=False, sep='|')
test_df.to_csv('test_data.csv', index=False, sep='|')

In [ ]:
train_df.to_json('train_data.json')
val_df.to_json('val_data.json')
test_df.to_json('test_data.json')

In [ ]:
# !pip install torchtext==0.10.1

### HyperParams

In [3]:
import torch

/home/csgrad/jayashok/miniconda3/envs/fact_hallucination/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MAX_VOCAB_SIZE = 12000
BATCH_SIZE = 64
EMBEDDING_DIM = 300
ENC_LAYERS = 2
ENC_HIDDEN_DIM = 512
DEC_HIDDEN_DIM = 512
DEC_LAYERS = 4
DROPOUT = 0.6

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("We are working with", DEVICE)

We are working with cuda:0


### TorchText Data Loader

In [5]:
# import pandas as pd
import torchtext.legacy as tt
import spacy

In [6]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
# Run First Time
# !python -m spacy download en_core_web_sm

In [8]:
# Fields
TEXT = tt.data.Field(
                    #sequential=True,
                     tokenize = 'spacy',
                     init_token = '<bos>',
                     eos_token = '<eos>',
                    tokenizer_language="en_core_web_sm",
                    lower=True # Can try this to reduce vocabulary size?
                    )

fields = [('dialog_idx', None), ('response', TEXT), ('original_response', None), ('history', None), ('knowledge', TEXT), ('BEGIN', None), ('VRM', None), ('prompt', TEXT)]

# Tabular Dataset creation
train_data, val_data, test_data = tt.data.TabularDataset.splits(
    path = "./",
    train="train_data.csv",
    validation="val_data.csv",
    test="test_data.csv",
    format="csv",
    skip_header=True,
    csv_reader_params=dict(delimiter='|'),
    fields=fields)



In [9]:
len(train_data.examples), len(val_data.examples)

(18357, 3417)

In [12]:
TEXT.build_vocab(train_data, min_freq=2, max_size=MAX_VOCAB_SIZE, vectors = "fasttext.en.300d",
                 unk_init = torch.Tensor.normal_)

In [13]:
len(TEXT.vocab)

12004

In [14]:
train_iterator, val_iterator, test_iterator = tt.data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    shuffle=True,
    #: # sort_within_batch = True, #Not applicable here mostly
    device = DEVICE)

### Define Model

In [15]:
import torch.nn as nn
import random

class EncoderLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        # super(EncoderLSTM, self).__init__()
        super().__init__()
        self.n_layers = n_layers
        self.directions = 2 if bidirectional else 1
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional, batch_first=False)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc_hidden = nn.Linear(self.directions*hidden_dim, hidden_dim)
        self.fc_cell = nn.Linear(self.directions*hidden_dim, hidden_dim)
        
        #4. Initialize Dropout
        self.dropout_in = nn.Dropout(p=dropout)
        self.dropout_out = nn.Dropout(p=dropout)
        
    def forward(self, text):
        #text = [sent_len, batch_size]
        
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        text_embed = self.embedding(text)
        # print('embedding:', text_embed.shape)
        rnn_input = self.dropout_in(text_embed)
        # rnn_input = nn.utils.rnn.pack_padded_sequence(text_embed, text_lengths.cpu())
        # rnn_input = text_embed

        #2. Run the RNN along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions];#hidden = [num layers * num directions, batch size, hid dim]
        rnn_out, (hidden, cell) = self.lstm(rnn_input)
        
        # Keep hidden state from last layer
        if self.directions == 2:
            # 2*n_layers * B * C
            hid_dim = hidden.shape[-1]
            hidden = hidden.view(self.n_layers, 2, -1, hid_dim)
            hidden = torch.cat([hidden[-1, 0, :, :], hidden[-1, 1, :, :]], dim=-1)
        else:
            # n_layers * B * C
            hidden = hidden[-1, :, :]

        hidden = self.dropout_out(hidden)

        # Keep cell state from last layer
        if self.directions == 2:
            # 2*n_layers * B * C
            cell_dim = cell.shape[-1]
            cell = cell.view(self.n_layers, 2, -1, cell_dim)
            cell = torch.cat([cell[-1, 0, :, :], cell[-1, 1, :, :]], dim=-1)
        else:
            # n_layers * B * C
            cell = cell[-1, :, :]

        # print('cell:', cell.shape)
        cell = self.dropout_out(cell)            

        return torch.nn.functional.relu(self.fc_hidden(hidden)), torch.nn.functional.relu(self.fc_cell(cell))
    

class DecoderLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, dropout, pad_idx):
        
        super().__init__()

        # self.dropout = dropout
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=False, dropout=dropout)
        
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim, vocab_size)

        #4. Initialize Dropout
        self.dropout_in = nn.Dropout(p=dropout)
        # self.dropout_out = nn.Dropout(p=dropout)
        
    def forward(self, text, hidden, cell):
        #text = [sent_len, batch_size]
        
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        text_embed = self.embedding(text)
        # print('got decoder input embeddings:', text_embed.shape)
        rnn_input = self.dropout_in(text_embed)
        # print('Completed Dropout')
        # rnn_input = nn.utils.rnn.pack_padded_sequence(text_embed, text_lengths.cpu())
        # rnn_input = text_embed
        
        #2. Run the RNN along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions];#hidden = [num layers * num directions, batch size, hid dim]        
        rnn_out, (hidden, cell) = self.lstm(rnn_input, (hidden, cell))
        # print('Done with LSTM')
        
        out = self.fc(rnn_out)
        
        return out, (hidden, cell)

In [16]:
class DialogGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout, enc_hidden_dim, enc_layers, dec_hidden_dim, dec_layers, pad_idx):
        
        super().__init__()
        self.vocab_size = vocab_size
        self.dec_layers = dec_layers

        self.enc_k = EncoderLSTM(vocab_size=vocab_size, 
                    embedding_dim=embedding_dim, 
                    hidden_dim=enc_hidden_dim,  
                    n_layers=enc_layers, 
                    bidirectional=True, 
                    dropout=dropout,
                    pad_idx=pad_idx)

        # self.enc_h = EncoderLSTM(vocab_size=vocab_size, 
        #             embedding_dim=embedding_dim, 
        #             hidden_dim=hidden_dim, 
        #             n_layers=enc_layers, 
        #             bidirectional=True, 
        #             dropout=dropout,
        #             pad_idx=pad_idx)

        self.enc_p = EncoderLSTM(vocab_size=vocab_size, 
                    embedding_dim=embedding_dim, 
                    hidden_dim=enc_hidden_dim, 
                    n_layers=enc_layers, 
                    bidirectional=True, 
                    dropout=dropout,
                    pad_idx=pad_idx)
        
        self.decoder = DecoderLSTM(vocab_size=vocab_size, 
                    embedding_dim=embedding_dim, 
                    hidden_dim=dec_hidden_dim, 
                    n_layers=dec_layers, 
                    dropout=dropout,
                    pad_idx=pad_idx)
        
        self.dropout = nn.Dropout(0.2)
        # self.fc_kh = nn.Linear(2*hidden_dim, hidden_dim)
        self.fc_kp_hidden = nn.Linear(2*enc_hidden_dim, dec_hidden_dim)
        self.fc_kp_cell = nn.Linear(2*enc_hidden_dim, dec_hidden_dim)

    def forward(self, knowledge, prompt, target_resp, teacher_forcing_ratio=0.5):
        # h = self.enc_h(history)
        k_hidden, k_cell = self.enc_k(knowledge)
        p_hidden, p_cell = self.enc_p(prompt)

        # k_hidden = self.enc_k(knowledge)
        # p_hidden = self.enc_p(prompt)

        # print('k-encoded:', k_hidden.shape, k_cell.shape)
        # print('p-encoded:', p_hidden.shape, p_cell.shape)

        kp_hidden = torch.cat([k_hidden, p_hidden], dim=-1)
        kp_cell = torch.cat([k_cell, p_cell], dim=-1)
        
        kp_hidden = self.dropout(kp_hidden)
        kp_cell = self.dropout(kp_cell)

        context_hidden = torch.nn.functional.relu(self.fc_kp_hidden(kp_hidden))
        context_cell = torch.nn.functional.relu(self.fc_kp_cell(kp_cell))

        # print('context_hidden:', context_hidden.shape)
        # print('context_cell:', context_cell.shape)
        
        # Stack hidden/cell states to be equal to the number of decoder layers
        # eg. (B x H) becomes (self.dec_layers x B x H)
        # if self.dec_layers > 1:
        context_hidden = torch.stack([context_hidden]*self.dec_layers, dim=0)
        context_cell = torch.stack([context_cell]*self.dec_layers, dim=0)
        # context_cell = torch.zeros_like(context_hidden)
        
        # Target response => TxB format
        target_seq_len = target_resp.size(0)
        batch_size = target_resp.size(1)
        
        # Outputs => format => T x B x C
        predictions = torch.zeros(target_seq_len, batch_size, self.vocab_size).to(DEVICE)

        # First Token in Target Response => '<bos>'
        input_at_t = target_resp[0, :]
        # print('first input shape:', input_at_t.shape)

        # Start from t=1, cuz first is output is '<bos>'
        for t in range(1, target_seq_len):
            
            # print('input at t:', input_at_t.shape)
            output_at_t, (context_hidden, context_cell) = self.decoder(torch.unsqueeze(input_at_t, axis=0), context_hidden, context_cell)
            # print('1 step decoder - lstm:', output.shape)
            
            output_at_t = torch.squeeze(output_at_t, axis=0)
            # print('output at t:', output_at_t.shape)
            predictions[t, :, :] = output_at_t
            
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing:
                # print('teacher forcing = True')
                input_at_t = target_resp[t, :]
            else:
                input_at_t = output_at_t.argmax(dim=1)
        
        return predictions

In [20]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
PAD_IDX

1

In [21]:
print(TEXT.init_token, TEXT.eos_token, TEXT.unk_token, TEXT.pad_token)
len(TEXT.vocab)

<bos> <eos> <unk> <pad>


12004

In [22]:
model = DialogGenerator(
     vocab_size=len(TEXT.vocab),
     embedding_dim=EMBEDDING_DIM,
     dropout=DROPOUT,
     enc_layers=ENC_LAYERS,
     enc_hidden_dim=ENC_HIDDEN_DIM,
     dec_layers=DEC_LAYERS,
     dec_hidden_dim=DEC_HIDDEN_DIM,
     pad_idx=PAD_IDX
)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 47,348,852 trainable parameters


In [24]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12004, 300])


In [25]:
model.enc_k.embedding.weight.data.copy_(pretrained_embeddings)
model.enc_p.embedding.weight.data.copy_(pretrained_embeddings)
model.decoder.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.1020,  1.6282,  2.1635,  ..., -0.6009, -0.1467,  0.0285],
        ...,
        [-0.4756, -0.1649,  0.0432,  ...,  0.6054,  0.4178,  0.2013],
        [-0.2901,  0.1146, -0.2877,  ...,  0.0295, -0.1209,  0.2231],
        [ 0.3811, -0.4038, -0.2008,  ..., -0.1453,  0.2882, -0.2616]])

In [26]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.enc_k.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.enc_p.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.decoder.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)

model.enc_k.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model.enc_p.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model.decoder.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.decoder.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1020,  1.6282,  2.1635,  ..., -0.6009, -0.1467,  0.0285],
        ...,
        [-0.4756, -0.1649,  0.0432,  ...,  0.6054,  0.4178,  0.2013],
        [-0.2901,  0.1146, -0.2877,  ...,  0.0295, -0.1209,  0.2231],
        [ 0.3811, -0.4038, -0.2008,  ..., -0.1453,  0.2882, -0.2616]])


## Train the Model

In [26]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=3e-4)

In [27]:
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

The criterion function calculates the loss, however we have to write our own function to calculate the accuracy. 

This function first feeds the predictions through a sigmoid layer, squashing the values between 0 and 1, we then round them to the nearest integer. This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).

We then calculate how many rounded predictions equal the actual labels and average it across the batch.

In [29]:
def train(model, iterator, optimizer, criterion, clip):
    
    epoch_loss = 0
    # epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        target_response = batch.response
        predictions = model(batch.knowledge, batch.prompt, target_response).squeeze(1)

        # Skip the first time-step in loss calculation...
        output_dim = predictions.shape[-1]
        predictions = predictions[1:].view(-1, output_dim)
        target_response = target_response[1:].view(-1)

        loss = criterion(predictions, target_response)

        # acc = binary_accuracy(predictions, batch.label)
        
        #calculate/accumulate gradients 
        loss.backward()

        # clip large gradients => to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Use Adam to update parameters based on gradient values
        optimizer.step()
        
        epoch_loss += loss.item()
        # epoch_acc += acc.item()
        
    return epoch_loss / len(iterator)

Then we define a function for evaluating our model, again remembering to separate `batch.text`.

`evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

`model.eval()` puts the model in "evaluation mode", this turns off _dropout_ and _batch normalization_.

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [27]:
import spacy

def infer(model, prompt, knowledge, max_words=50):
    nlp = spacy.load('en_core_web_sm')
    
    model.eval()
    p_tokens = [tok.text.lower() for tok in nlp.tokenizer(prompt)]
    k_tokens = [tok.text.lower() for tok in nlp.tokenizer(knowledge)]
    # print('tokens:', p_tokens, k_tokens)

    p_indexed = [TEXT.vocab.stoi[t] for t in p_tokens]
    k_indexed = [TEXT.vocab.stoi[t] for t in k_tokens]

    p_tensor = torch.LongTensor(p_indexed).to(DEVICE).unsqueeze(1)
    k_tensor = torch.LongTensor(k_indexed).to(DEVICE).unsqueeze(1)

    # print(p_tensor.shape, k_tensor.shape)

    prediction = model(k_tensor, p_tensor, torch.zeros(max_words, 1, dtype=torch.int).to(DEVICE), teacher_forcing_ratio=0).argmax(dim=-1).squeeze().cpu().numpy()
    
    gen_response = []
    for pred_idx in list(prediction):
        if pred_idx > max_words:
            break
        
        if pred_idx==TEXT.vocab.stoi['<eos>']:
            break
        gen_response.append(TEXT.vocab.itos[pred_idx])

    # response = " ".join(gen_response)
    return gen_response

In [29]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    # epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            target_response = batch.response
            predictions = model(batch.knowledge, batch.prompt, target_response, teacher_forcing_ratio=0).squeeze(1)
            
            # return target_response, predictions
            # Skip the first time-step in loss calculation...
            output_dim = predictions.shape[-1]
            predictions = predictions[1:].view(-1, output_dim)
            target_response = target_response[1:].view(-1)

            loss = criterion(predictions, target_response)

            epoch_loss += loss.item()

    
        response = infer(model, prompt="I really like the color red. I find it beautiful.",
                          knowledge="It has a dominant wavelength of approximately 625–740 nanometres.",
                          max_words=25)
        print('response:', response)
    return epoch_loss / len(iterator)

In [33]:
evaluate(model, val_iterator, criterion)

response: ['<unk>']


8.990068241401955

And also create a nice function to tell us how long our epochs are taking.

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We then train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model on the mounted gdrive, and then after training has finished we'll use that model on the test set. You can see the saved model in the left panel. Click on the folder option and go to gdrive -> My Drive. 

**Note**: you are required to submit this "rnn_model.pt" file for grading.

In [38]:
import math

N_EPOCHS = 50
CLIP = 1
# path = F"/content/gdrive/My Drive/rnn_model.pt"
path = "exp_12.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, clip=CLIP)
    valid_loss = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\tVal. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

response: ['<unk>', 'i', ',']
Epoch: 01 | Time: 0m 56s
	Train Loss: 5.965 | Train PPL: 389.470
	Val. Loss: 5.735 |  Val. PPL: 309.476
response: ['<unk>', 'i', ',', 'that', ',', ',', '<unk>', '.']
Epoch: 02 | Time: 0m 57s
	Train Loss: 5.785 | Train PPL: 325.454
	Val. Loss: 5.717 |  Val. PPL: 304.135
response: ['<unk>', 'i', ',', ',', 'know', 'the', 'the', 'the', 'the', 'the']
Epoch: 03 | Time: 0m 57s
	Train Loss: 5.773 | Train PPL: 321.661
	Val. Loss: 5.714 |  Val. PPL: 303.016
response: ['<unk>', 'i', ',', 'the', 'the', 'the', 'the', 'the', 'the']
Epoch: 04 | Time: 0m 57s
	Train Loss: 5.767 | Train PPL: 319.585
	Val. Loss: 5.711 |  Val. PPL: 302.063
response: ['<unk>', 'i', ',', ',', 'the', 'the', 'the', 'the']
Epoch: 05 | Time: 0m 57s
	Train Loss: 5.764 | Train PPL: 318.471
	Val. Loss: 5.709 |  Val. PPL: 301.511
response: ['<unk>', 'i', ',', ',', 'know', ',', ',', ',', ',', ',', '.']
Epoch: 06 | Time: 0m 57s
	Train Loss: 5.761 | Train PPL: 317.558
	Val. Loss: 5.708 |  Val. PPL: 301.22

In [40]:
best_valid_loss

5.700669641847964

### Load Pretrained Model

In [30]:
model.load_state_dict(torch.load("exp_9.pt"))

<All keys matched successfully>

### Evaluating the model

In [30]:
!pip install rouge

In [31]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 746.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.3 MB/s eta 0:00:0000:0100:01
  Using cached matplotlib-3.7.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 9.4 MB/s eta 0:00:0000:01
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.2 MB/s eta 0:00:0000:0100:01
  Using cached kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.8 MB/s eta 0:00:00:00:01
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
  Using cached contourpy-1.0.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Usin

In [ ]:
#passing tokens to all of the models directly

In [33]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.8 MB/s eta 0:00:0000:01


In [40]:
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
import torch
from bert_score import score
from rouge import Rouge

In [41]:
rouge = Rouge()

In [42]:
def bleu_eval(p, g):
    bleu_score = sentence_bleu([g], p)
    # print("BLEU score:", bleu_score)
    return bleu_score

# bleu_eval(predicted_sentence, ground_truth_sentence)

In [43]:
def rouge_eval(p, g):    
    scores = rouge.get_scores(" ".join(p), " ".join(g))
    # print("ROUGE-1 score:", scores[0]['rouge-1']['f'])
    # print("ROUGE-2 score:", scores[0]['rouge-2']['f'])
    return scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']
  # rouge_eval(predicted_sentence, ground_truth_sentence)

In [44]:
def bert_eval(p, g):
    P, R, F1 = score([' '.join(p)], [' '.join(g)], lang='en', verbose=False)
    # print("BERT score (F1):", F1.item())
    return F1.item()
# bert_eval(predicted_sentence, ground_truth_sentence)

In [45]:
from tqdm import tqdm

In [46]:
def test(model, iterator):
    results = []
    model.eval()    
    with torch.no_grad():
        for batch in tqdm(iterator, total=len(iterator)):
            target_response = batch.response
            predictions = model(batch.knowledge, batch.prompt, target_response, teacher_forcing_ratio=0).squeeze(1).cpu().numpy()
            # Skip the first time-step
            batch_size = predictions.shape[-2]
            predictions = predictions[1:].argmax(-1).transpose(1, 0)
            target_response = target_response[1:].transpose(1, 0)
            # return predictions, target_response
            results.extend(get_scores(predictions, target_response))
        # print('response:', response)
    return results

In [47]:
def greedy_decode(prediction, max_words=50):
    gen_response = []
    for pred_idx in list(prediction):
        if len(gen_response) > max_words:
            break
        if pred_idx==TEXT.vocab.stoi['<eos>']:
            break
        gen_response.append(TEXT.vocab.itos[pred_idx])
    return gen_response

In [48]:
def get_scores(pred_batch, gt_batch):
    scores = []
    batch_size = len(pred_batch)
    for idx in range(batch_size):
        pred_tokens = greedy_decode(pred_batch[idx])
        gt_tokens = greedy_decode(gt_batch[idx])
        # print(pred)
        # return pred_tokens, gt_tokens
        bleu = bleu_eval(pred_tokens, gt_tokens)
        rouge1, rouge2, rougel = rouge_eval(pred_tokens, gt_tokens)
        # bert_f1 = bert_eval(pred_tokens, gt_tokens)
        scores.append({
                'pred': " ".join(pred_tokens),
                'gt': " ".join(gt_tokens),
                'bleu': bleu,
                'rouge1': rouge1, 
                'rouge2': rouge2,
                'rougel': rougel,
                # 'bert_f1': bert_f1
            })

    return scores

In [49]:
model.to(DEVICE)

DialogGenerator(
  (enc_k): EncoderLSTM(
    (embedding): Embedding(12004, 300, padding_idx=1)
    (lstm): LSTM(300, 512, num_layers=2, bidirectional=True)
    (fc_hidden): Linear(in_features=1024, out_features=512, bias=True)
    (fc_cell): Linear(in_features=1024, out_features=512, bias=True)
    (dropout_in): Dropout(p=0.6, inplace=False)
    (dropout_out): Dropout(p=0.6, inplace=False)
  )
  (enc_p): EncoderLSTM(
    (embedding): Embedding(12004, 300, padding_idx=1)
    (lstm): LSTM(300, 512, num_layers=2, bidirectional=True)
    (fc_hidden): Linear(in_features=1024, out_features=512, bias=True)
    (fc_cell): Linear(in_features=1024, out_features=512, bias=True)
    (dropout_in): Dropout(p=0.6, inplace=False)
    (dropout_out): Dropout(p=0.6, inplace=False)
  )
  (decoder): DecoderLSTM(
    (embedding): Embedding(12004, 300, padding_idx=1)
    (lstm): LSTM(300, 512, num_layers=4, dropout=0.6)
    (fc): Linear(in_features=512, out_features=12004, bias=True)
    (dropout_in): Dropou

In [50]:
test_results = test(model, test_iterator)

  0%|          | 0/56 [00:00<?, ?it/s]/home/csgrad/jayashok/miniconda3/envs/fact_hallucination/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/csgrad/jayashok/miniconda3/envs/fact_hallucination/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/csgrad/jayashok/miniconda3/envs/fact_hallucination/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BL

In [51]:
import pandas as pd

In [52]:
df = pd.DataFrame(test_results)

In [54]:
df.to_csv('exp_9_results.csv', index=False)

In [56]:
df.rouge2.mean()

0.005471364832767729

In [132]:
df.bleu.mean()

9.295372088924811e-82

In [134]:
df.rouge1.mean()

0.19879693109538177

In [135]:
df.rougel.mean()

0.17766316792990272

In [136]:
df

,pred,gt,bleu,rouge1,rouge2,rougel
0,"i , not you , that the the the the the the .","oh wow , really ? skiing can be a means of tra...",3.669559e-232,0.133333,0.0,0.133333
1,"i , not you , that the the the the the the .",did you know a drum kit 's classical <unk> inc...,1.002492e-231,0.210526,0.0,0.210526
2,"i , not you , that the the the the the the .",i do n't have any likes for music as a bot . d...,3.224148e-232,0.171429,0.0,0.171429
3,"i , not you , that the the the the the the .",some people keep bees in man - made <unk> . it...,7.766827e-232,0.095238,0.0,0.095238
4,"i , not you , that the the the the the the .",a <unk> is the result of breeding a golden ret...,7.959013e-232,0.190476,0.0,0.190476
...,...,...,...,...,...,...
3534,"i , not you , that the the the the the the .","yes , it was adapted into a film in 1993 , sta...",1.082650e-231,0.105263,0.0,0.105263
3535,"i , not you , that the the the the the the .","i would n't know , but there are actually diff...",5.821803e-232,0.148148,0.0,0.148148
3536,"i , not you , that the the the the the the .","no , i have n't been there - i understand that...",8.373637e-232,0.333333,0.0,0.250000
3537,"i , not you , that the the the the the the .",in the rgb system it is a mixture of <unk> and...,1.140985e-231,0.111111,0.0,0.111111


#### Get Task1 Model Results

In [35]:
import importlib

In [62]:
import task1_infer

In [57]:
# generate knowledge, pred_response-pairs

In [66]:
importlib.reload(task1_infer)

Unique tokens in KNOWLEDGE vocabulary: 24539
Unique tokens in RESPONSE vocabulary: 25002
Unique tokens in HISTORY vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


<module 'task1_infer' from '/home/csgrad/jayashok/fact_hallucination/task1_infer.py'>

In [69]:
scores = []
for batch in test_iterator:
    pred_resp = ' '.join(['i', ',', 'not', 'you', ',', 'that', 'the', 'the', 'the', 'the', 'the', 'the', '.'])
    knowledge = batch.knowledge.transpose(1, 0)
    prompt = batch.prompt.transpose(1, 0)
    for idx in range(knowledge.shape[0]):
        k_decoded = ' '.join(greedy_decode(knowledge[idx]))
        h_decoded = ' '.join(greedy_decode(prompt[idx]))
        res = task1_infer.predict_hallucination(task1_infer.model, k_decoded, h_decoded, pred_resp)
        scores.append(res)

In [70]:
import numpy as np

In [71]:
scores = np.array(scores)

In [75]:
np.median(scores)

0.2024255394935608

In [77]:
scores.std()

0.009732419186018566

In [73]:
scores.mean()

0.20215770788332876

In [68]:
scores

0.2091849148273468